# 01 Data Ingestion & Preprocessing

## 1. Imports

In [1]:
import pandas as pd
import numpy as np

# Pfade
RAW_DIR = "../data/raw"
PROCESSED_DIR = "../data/processed"

## 2. Daten einlesen

In [2]:
cars = pd.read_csv(f"{RAW_DIR}/car_prices.csv", parse_dates=["saledate"])
states = pd.read_csv(f"{RAW_DIR}/states.csv")

/tmp/ipykernel_39668/3268363706.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cars = pd.read_csv(f"{RAW_DIR}/car_prices.csv", parse_dates=["saledate"])


## 3. Erste Übersicht

In [3]:
print("Cars:", cars.shape)
print("States:", states.shape)
display(cars.head(), states.head())

Cars: (558837, 16)
States: (51, 4)


,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


## 4. Datentypen & Datumskonvertierung

In [4]:
# 4.1 Prüfen der aktuellen Datentypen
print(cars.dtypes)

# 4.2 saledate in datetime64 umwandeln (mit UTC, dann Zeitzone entfernen)
cars["saledate"] = pd.to_datetime(
    cars["saledate"],
    errors="coerce",
    utc=True
)
cars["saledate"] = cars["saledate"].dt.tz_convert(None)

# 4.3 Kontrolle: Datentyp und Anzahl fehlender Datumswerte
print("Neuer dtype:", cars["saledate"].dtype)
print("Anzahl NaT-Werte:", cars["saledate"].isna().sum())

# 4.4 Optional: Zeilen mit ungültigen Datumsangaben anzeigen
bad_dates = cars[cars["saledate"].isna()]
display(bad_dates.head())

# 4.5 Datumskomponenten extrahieren (funktioniert jetzt ohne Fehler)
cars["sale_year"]    = cars["saledate"].dt.year
cars["sale_month"]   = cars["saledate"].dt.month
cars["sale_day"]     = cars["saledate"].dt.day       # Tag des Monats
cars["sale_weekday"] = cars["saledate"].dt.weekday   # Wochentag (0=Montag)

# 4.6 Vorschau der neuen Spalten
display(cars[["saledate", "sale_year", "sale_month", "sale_day", "sale_weekday"]].head())


year              int64
make             object
model            object
trim             object
body             object
transmission     object
vin              object
state            object
condition       float64
odometer        float64
color            object
interior         object
seller           object
mmr             float64
sellingprice    float64
saledate         object
dtype: object


/tmp/ipykernel_39668/1200531906.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cars["saledate"] = pd.to_datetime(


Neuer dtype: datetime64[ns]
Anzahl NaT-Werte: 38


,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
306447,2013,Hyundai,Sonata,GLS,Sedan,automatic,5npeb4ac6dh687932,il,19.0,37254.0,gray,gray,kfl llc,NaN,NaN,NaT
406524,2013,Chrysler,300,Base,Sedan,automatic,2c3ccaag9dh723146,il,27.0,44208.0,silver,black,kfl llc,NaN,NaN,NaT
408161,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj4fm201708,NaN,46.0,4802,silver,gray,NaN,13200.0,NaT
417835,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj2fm258506,NaN,1.0,9410,white,gray,NaN,13300.0,NaT
421289,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj3fm276741,NaN,46.0,1167,blue,black,NaN,13200.0,NaT


,saledate,sale_year,sale_month,sale_day,sale_weekday
0,2014-12-16 04:30:00,2014.0,12.0,16.0,1.0
1,2014-12-16 04:30:00,2014.0,12.0,16.0,1.0
2,2015-01-14 20:30:00,2015.0,1.0,14.0,2.0
3,2015-01-28 20:30:00,2015.0,1.0,28.0,2.0
4,2014-12-18 04:30:00,2014.0,12.0,18.0,3.0


In [5]:
# 4.7 Kontrollieren, ob noch NaT-Werte vorhanden sind
print("NaT in 'saledate':", cars["saledate"].isna().sum())
cars.loc[cars["saledate"].isna(), :].head(5)

NaT in 'saledate': 38


,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,sale_year,sale_month,sale_day,sale_weekday
306447,2013,Hyundai,Sonata,GLS,Sedan,automatic,5npeb4ac6dh687932,il,19.0,37254.0,gray,gray,kfl llc,NaN,NaN,NaT,NaN,NaN,NaN,NaN
406524,2013,Chrysler,300,Base,Sedan,automatic,2c3ccaag9dh723146,il,27.0,44208.0,silver,black,kfl llc,NaN,NaN,NaT,NaN,NaN,NaN,NaN
408161,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj4fm201708,NaN,46.0,4802,silver,gray,NaN,13200.0,NaT,NaN,NaN,NaN,NaN
417835,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj2fm258506,NaN,1.0,9410,white,gray,NaN,13300.0,NaT,NaN,NaN,NaN,NaN
421289,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj3fm276741,NaN,46.0,1167,blue,black,NaN,13200.0,NaT,NaN,NaN,NaN,NaN


## 5. Merge mit States

### Überprüfung der State-Codes

In [6]:
# Schritt A: Überprüfung der State-Codes vor dem Merge

# A.1 Einzigartige State-Codes im Auto-DataFrame
unique_car_states = cars["state"].unique()
print("Einzigartige State-Codes in cars:", unique_car_states)

# A.2 Einzigartige State-Codes im states-DataFrame
unique_states = states["State Code"].unique()
print("Einzigartige State-Codes in states:", unique_states)

# A.3 Gemeinsame und fehlende Codes
common_states = set(unique_car_states).intersection(unique_states)
missing_in_states = set(unique_car_states) - set(unique_states)
print("\nGemeinsame Codes:", sorted(common_states))
print("Codes in cars, aber nicht in states:", sorted(missing_in_states))

# A.4 Optional: Häufigkeit der fehlenden Codes in cars
if missing_in_states:
    freq_missing = cars.loc[cars["state"].isin(missing_in_states), "state"].value_counts()
    print("\nHäufigkeit fehlender Codes in cars:")
    print(freq_missing)



Einzigartige State-Codes in cars: ['ca' 'tx' 'pa' 'mn' 'az' 'wi' 'tn' 'md' 'fl' 'ne' 'nj' 'nv' 'oh' 'mi'
 'ga' 'va' 'sc' 'nc' 'in' 'il' 'co' 'ut' 'mo' 'ny' 'ma' 'pr' 'or' 'la'
 'wa' 'hi' 'qc' 'ab' 'on' 'ok' 'ms' 'nm' 'al' '3vwd17aj4fm201708' 'ns'
 '3vwd17aj2fm258506' '3vwd17aj3fm276741' '3vwd17aj2fm285365'
 '3vwd17aj0fm227318' '3vwd17aj6fm218641' '3vwd17aj7fm223475'
 '3vwd17aj5fm297123' '3vwd17aj5fm219943' '3vwd17aj9fm219766'
 '3vwd17aj3fm259017' '3vwd17aj5fm206111' '3vwd17aj5fm273601'
 '3vwd17aj5fm221322' '3vwd17aj5fm268964' '3vwd17aj6fm231972'
 '3vwd17aj7fm222388' '3vwd17aj7fm218440' '3vwd17ajxfm315938'
 '3vwd17aj7fm229552' '3vwd17aj8fm298895' '3vwd17aj4fm236636'
 '3vwd17aj5fm225953' '3vwd17aj7fm326640' '3vwd17aj8fm239622'
 '3vwd17aj2fm261566']
Einzigartige State-Codes in states: ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID'
 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC'
 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC'

### 5.2 State-Codes vereinheitlichen

In [7]:
# Schritt A: State-Codes bereinigen

# A.1 Uppercase
cars["state_clean"] = cars["state"].str.upper()

# A.2 Ungültige Codes auf NaN setzen
# Zulässige Codes sind genau die 2-stelligen aus states["State Code"]
valid_codes = set(states["State Code"])
mask_valid = cars["state_clean"].str.len() == 2
cars.loc[~mask_valid, "state_clean"] = pd.NA

# A.3 Codes, die nicht in valid_codes sind, ebenfalls auf NaN
mask_not_in_list = ~cars["state_clean"].isin(valid_codes)
cars.loc[mask_not_in_list, "state_clean"] = pd.NA

# A.4 Überblick über Bereinigung
print("Ursprüngliche unique codes:", cars["state"].nunique())
print("Bereinigte unique codes:", cars["state_clean"].nunique())
print("Anzahl ungültiger codes:", cars["state_clean"].isna().sum())
print("Beispiele gültiger Codes:", cars["state_clean"].dropna().unique()[:10])

# Schritt B: Erneute Überprüfung
valid_in_cars = set(cars["state_clean"].dropna().unique())
print("Codes in cars_clean ∩ states:", sorted(valid_in_cars & valid_codes))



Ursprüngliche unique codes: 64
Bereinigte unique codes: 33
Anzahl ungültiger codes: 8427
Beispiele gültiger Codes: ['CA' 'TX' 'PA' 'MN' 'AZ' 'WI' 'TN' 'MD' 'FL' 'NE']
Codes in cars_clean ∩ states: ['AL', 'AZ', 'CA', 'CO', 'FL', 'GA', 'HI', 'IL', 'IN', 'LA', 'MA', 'MD', 'MI', 'MN', 'MO', 'MS', 'NC', 'NE', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'SC', 'TN', 'TX', 'UT', 'VA', 'WA', 'WI']


### 5.3 Ungültige State-codes analysieren

In [8]:
# A. Ungültige Codes isolieren
invalid_states = cars.loc[cars["state_clean"].isna(), "state"]

# B. Einzigartige ungültige Werte und ihre Häufigkeit
invalid_counts = invalid_states.value_counts()

# C. Übersicht ausgeben
print("Anzahl ungültiger Einträge insgesamt:", len(invalid_states))
print("Anzahl verschiedener ungültiger Codes:", invalid_counts.shape[0])

display(invalid_counts.head(20))   # Top 20 der ungültigen Codes


Anzahl ungültiger Einträge insgesamt: 8427
Anzahl verschiedener ungültiger Codes: 31


state
on                   3442
pr                   2725
qc                   1245
ab                    928
ns                     61
3vwd17aj4fm201708       1
3vwd17aj2fm258506       1
3vwd17aj3fm276741       1
3vwd17aj2fm285365       1
3vwd17aj0fm227318       1
3vwd17aj6fm218641       1
3vwd17aj7fm223475       1
3vwd17aj5fm297123       1
3vwd17aj5fm219943       1
3vwd17aj9fm219766       1
3vwd17aj3fm259017       1
3vwd17aj5fm206111       1
3vwd17aj5fm273601       1
3vwd17aj5fm221322       1
3vwd17aj5fm268964       1
Name: count, dtype: int64

## Erläuterung 

Ich habe das erste Notebook vorläufig abgebrochen, weil sich herausgestellt hat, dass die zugrunde liegenden State-Daten nicht vollständig waren – die ursprüngliche Datei enthielt nur US-Staaten, ohne die kanadischen Provinzen. Um eine saubere und reproduzierbare Pipeline aufzubauen, möchte ich jetzt von Grund auf neu starten und direkt mit der aktualisierten „USA + Kanada“-Tabelle arbeiten. So stelle ich sicher, dass alle State-Codes korrekt geparst und gemergt werden, bevor ich mit weiteren Preprocessing- und Feature-Engineering-Schritten fortfahre.
- 21.05.2025